In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
# Period 2018: [2018-08-11, 2018-11-03]


In [3]:
sales_all=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-06-16.csv",dtype=str)
sales_all['sales']=sales_all['sales'].astype(float)
sales_all['week_end_date']=sales_all['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_all_2017_Q3=sales_all[(sales_all['week_end_date']>=datetime.date(2018,8,11)-datetime.timedelta(days=52*7)) &\
                           (sales_all['week_end_date']<=datetime.date(2018,11,3)-datetime.timedelta(days=52*7))]
sales_all_2017_Q3=sales_all_2017_Q3[sales_all_2017_Q3['sales']>0]
US_Sales=sales_all_2017_Q3['sales'].sum()

In [4]:
Store_DMA=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA_20180620.xlsx",dtype=str)
Store_DMA=Store_DMA[['location_id','DMA']].drop_duplicates()

In [5]:
sales_all_2017_Q3=pd.merge(sales_all_2017_Q3,Store_DMA,on="location_id",how="left")

In [6]:
sales_all_2017_Q3_DMA=sales_all_2017_Q3.groupby(['DMA'])['sales'].sum().to_frame().reset_index()

In [7]:
sales_all_2017_Q3_DMA.shape

(186, 2)

In [8]:
all_zips=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/BL_All Zips with Q2 Label_JL_20180402 new label.xlsx",dtype=str)

del all_zips['Store 2017 Q2 Sales Label']

Store_List_From_Dom=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/Store List 6.14.18 225PM.xlsx",dtype=str,sheetname="Store List")
Store_List_From_Dom=Store_List_From_Dom.iloc[0:1415,:]
# scheduled_close_store=['213','430','575','1233']
# Store_List_From_Dom=Store_List_From_Dom[~Store_List_From_Dom['Store'].isin(scheduled_close_store)]
# Keep the scheduled stores
Store_List_From_Dom=Store_List_From_Dom[['Store','City','ST','Open Date']]

Sales_Store=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-06-09.csv",dtype=str)
Sales_Store=Sales_Store[Sales_Store['location_id'].isin(Store_List_From_Dom['Store'])]
Sales_Store['week_end_date']=Sales_Store['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Sales_Store['sales']=Sales_Store['sales'].astype(float)
Sales_Store=Sales_Store[Sales_Store['sales']>0]


Sales_Store_Q3_2017=Sales_Store[(Sales_Store['week_end_date']>= datetime.date(2017,8,12)) & (Sales_Store['week_end_date']<= datetime.date(2017,11,4))]

Sales_Store_Count=Sales_Store_Q3_2017.groupby(['location_id'])['week_end_date'].count().to_frame().reset_index()
Sales_Store_Count_Not_13=Sales_Store_Count[Sales_Store_Count['week_end_date']!=13]

Sales_Store_Before_Q3_2017=Sales_Store[~Sales_Store['location_id'].isin(Sales_Store_Count_Not_13['location_id'])]
Sales_Store_Before_Q3_2017=Sales_Store_Before_Q3_2017[Sales_Store_Before_Q3_2017['week_end_date']<datetime.date(2017,8,12)]

New_List_1=['310','1964','4193']
Avg_List=['550','1053','1801']
New_List_2=Sales_Store_Count_Not_13[~Sales_Store_Count_Not_13['location_id'].isin(Avg_List+New_List_1)]['location_id'].tolist()
New_List_3=[x for x in Store_List_From_Dom['Store'] if x not in Sales_Store_Q3_2017['location_id'].unique().tolist()]
New_List=New_List_1+New_List_2+New_List_3
Sales_Store_Before_Q3_2017['location_id'].unique()

Sales_Store_Q3_2017=Sales_Store_Q3_2017.groupby(['location_id'])['sales'].sum().to_frame().reset_index()
Sales_Store_Q3_2017=Sales_Store_Q3_2017[~Sales_Store_Q3_2017['location_id'].isin(New_List)]
Sales_Store_Q3_2017=pd.merge(Sales_Store_Q3_2017,Sales_Store_Count,on="location_id",how="left")
Sales_Store_Q3_2017['sales_avg']=Sales_Store_Q3_2017['sales']/Sales_Store_Q3_2017['week_end_date']
Sales_Store_Q3_2017['store_sales_label_17Q3']=np.where(Sales_Store_Q3_2017['sales_avg']>=np.percentile(Sales_Store_Q3_2017['sales_avg'],80),"H",
                                            np.where(Sales_Store_Q3_2017['sales_avg']>=np.percentile(Sales_Store_Q3_2017['sales_avg'],20),"M","L")
                                            )
Sales_Store_Q3_2017_Label=Sales_Store_Q3_2017[['location_id','store_sales_label_17Q3']]

Store_List_From_Dom_Label=Store_List_From_Dom[['Store','City','ST','Open Date']].rename(columns={"Store":"location_id"})
Store_List_From_Dom_Label=pd.merge(Store_List_From_Dom_Label,Sales_Store_Q3_2017_Label,on="location_id",how="left")
Store_List_From_Dom_Label['store_sales_label_17Q3']=Store_List_From_Dom_Label['store_sales_label_17Q3'].fillna("new")
Store_List_From_Dom_Label=Store_List_From_Dom_Label[['location_id','store_sales_label_17Q3']]

all_zips_store=all_zips[all_zips['location_id']!="nan"]
all_zips_no_store=all_zips[all_zips['location_id']=="nan"]
all_zips_no_store['store_sales_label_17Q3']="nan"

all_zips_store=pd.merge(all_zips_store,Store_List_From_Dom_Label,on="location_id",how="left")

all_zips_store['store_sales_label_17Q3']=all_zips_store['store_sales_label_17Q3'].fillna("closed")
all_zips=all_zips_store.append(all_zips_no_store)

store_ta_label=all_zips[['location_id','allocated_dma_in_analysis','TA_of_store','store_sales_label_17Q3']]
store_ta_label=store_ta_label[~pd.isnull(store_ta_label['store_sales_label_17Q3'])]
store_ta_label=store_ta_label[store_ta_label['store_sales_label_17Q3']!='nan']
store_ta_label=store_ta_label.drop_duplicates().reset_index()
del store_ta_label['index']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
store_ta_label.head(2)

,location_id,allocated_dma_in_analysis,TA_of_store,store_sales_label_17Q3
0,1,"COLUMBUS, OH",N14,H
1,3,AUGUSTA-AIKEN,N83,M


In [10]:
store_ta_label_open=store_ta_label[store_ta_label['store_sales_label_17Q3']!="closed"]
store_ta_label_open=pd.merge(store_ta_label_open,Store_DMA,on="location_id",how="left")

store_count_open=store_ta_label_open.groupby(['DMA'])['location_id'].count().to_frame().reset_index()


store_ta_label_open_HMnew=store_ta_label_open[store_ta_label_open['store_sales_label_17Q3'].isin(['H','M','new'])]
store_count_HMnew=store_ta_label_open_HMnew.groupby(['DMA'])['location_id'].count().to_frame().reset_index()



In [11]:
sales_all_2017_Q3_HMnew=sales_all_2017_Q3[sales_all_2017_Q3['location_id'].isin(store_ta_label_open_HMnew['location_id'])]
sales_all_2017_Q3_HMnew=sales_all_2017_Q3_HMnew.groupby(['DMA'])['sales'].sum().to_frame().reset_index()
sales_all_2017_Q3_HMnew=sales_all_2017_Q3_HMnew.rename(columns={"sales":"H_M_new_sales"})

In [12]:
sales_all_2017_Q3_HMnew.head(2)

,DMA,H_M_new_sales
0,ABILENE-SWEETWATER,675165.01
1,"ALBANY, GA",733499.31


In [21]:
TA_count=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/Q3_2018/BL_zips_by_dma_by_ta_JL_2018-06-19.xlsx",sheetname="all_12_DMAs")
TA_count=TA_count[['DMA','TA']].drop_duplicates()
TA_count=TA_count.groupby(['DMA'])['TA'].count().to_frame().reset_index()

In [48]:
DMA_List=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/Q3_Markets.xlsx",dtype=str)
DMA_List.head(2)

,Market,Markets_Type
0,PHOENIX (PRESCOTT),Q3_DMA
1,FT. MYERS-NAPLES,Q3_DMA


In [58]:
output_table=DMA_List.rename(columns={"Market":"DMA","Markets_Type":"Previous_Campaign"})
output_table['Previous_Campaign']=np.nan

output_table=pd.merge(output_table,store_count_open,on="DMA",how="left")
output_table=output_table.rename(columns={"location_id":"Total_Stores"})

output_table=pd.merge(output_table,sales_all_2017_Q3_DMA,on="DMA",how="left")
output_table=output_table.rename(columns={"sales":"Total_2017_Q3_Sales"})
output_table['DMA (all_stores) Share of US']=output_table['Total_2017_Q3_Sales']/US_Sales

output_table=pd.merge(output_table,store_count_HMnew,on="DMA",how="left")
output_table=output_table.rename(columns={"location_id":"H/M/new store counts"})

output_table=pd.merge(output_table,TA_count,on="DMA",how="left")
output_table=output_table.rename(columns={"TA":"Trade Area Counts"})

output_table=pd.merge(output_table,sales_all_2017_Q3_HMnew,on="DMA",how="left")
output_table=output_table.rename(columns={"H_M_new_sales":"Total_DMA_Sales_of_H_M_new_stores"})
output_table['% Total DMA Sales']=output_table['Total_DMA_Sales_of_H_M_new_stores']/output_table['Total_2017_Q3_Sales']
output_table['DMA (H/M/new) Share of US']=output_table['Total_DMA_Sales_of_H_M_new_stores']/US_Sales

In [59]:
output_table

,DMA,Previous_Campaign,Total_Stores,Total_2017_Q3_Sales,DMA (all_stores) Share of US,H/M/new store counts,Trade Area Counts,Total_DMA_Sales_of_H_M_new_stores,% Total DMA Sales,DMA (H/M/new) Share of US
0,PHOENIX (PRESCOTT),NaN,26,19172729.14,0.017292,26,7,18818497.79,0.981524,0.016973
1,FT. MYERS-NAPLES,NaN,8,6329943.91,0.005709,7,2,5703506.30,0.901036,0.005144
2,BALTIMORE,NaN,16,13267892.79,0.011966,14,3,12063456.73,0.909222,0.010880
3,CHATTANOOGA,NaN,8,5550321.46,0.005006,6,3,4438051.08,0.799603,0.004003
4,SACRAMNTO-STKTON-MODESTO,NaN,20,18180277.47,0.016397,19,5,16981980.15,0.934088,0.015316
5,FRESNO-VISALIA,NaN,9,6420003.33,0.005790,7,4,5287412.98,0.823584,0.004769
6,MIAMI-FT. LAUDERDALE,NaN,13,14167665.18,0.012778,13,3,14167665.18,1.000000,0.012778
7,LAS VEGAS,NaN,9,8302259.20,0.007488,9,1,8302259.20,1.000000,0.007488
8,"COLUMBUS, OH",NaN,20,15280985.94,0.013782,14,4,11763460.53,0.769810,0.010610
9,CHAMPAIGN&SPRNGFLD-DECATUR,NaN,5,3880794.07,0.003500,4,4,3318829.34,0.855193,0.002993


In [67]:
Previous_Campaign=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q3/2017_Q3_Summary/Market_List_Till_Q3_2018.xlsx",dtype=str)
Labor_Day_DMAs=Previous_Campaign['Labor Day 20 Markets'].unique().tolist()
Labor_Day_DMAs.remove('nan')

October_DMAs=Previous_Campaign['Oct 29 Markets'].unique().tolist()
October_DMAs.remove('nan')

Holiday_DMAs=Previous_Campaign['Oct 29 Markets'].unique().tolist()

Q1_2018_DMAs=Previous_Campaign['2018 Q1 Markets'].unique().tolist()
Q1_2018_DMAs.remove('nan')

Q2_2018_DMAs=Previous_Campaign['2018_Q2 Markets'].unique().tolist()
Q2_2018_DMAs.remove('nan')

In [69]:
for i in range(len(output_table)):
    previous_campaign_list=[]
    if output_table['DMA'][i] in Labor_Day_DMAs:
        previous_campaign_list=previous_campaign_list+['LaborDay']
    if output_table['DMA'][i] in October_DMAs:
        previous_campaign_list=previous_campaign_list+['October']
    if output_table['DMA'][i] in Labor_Day_DMAs:
        previous_campaign_list=previous_campaign_list+['Holiday']
    if output_table['DMA'][i] in Q1_2018_DMAs:
        previous_campaign_list=previous_campaign_list+['Q1_2018']
    if output_table['DMA'][i] in Q2_2018_DMAs:
        previous_campaign_list=previous_campaign_list+['Q2_2018']
    output_table['Previous_Campaign'][i]=previous_campaign_list

        

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [71]:
output_table=output_table.sort_values(['DMA'])

In [73]:
output_table.to_csv("/home/jian/Projects/Big_Lots/Analysis/2018_Q3/2017_Q3_Summary/BL_Summary table for 2017 Q3_JL_20180625.csv")